In [1]:
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Rescaling, GlobalAveragePooling2D, Multiply
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
from scipy.spatial.distance import euclidean

In [2]:
# Define your data
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'archive/split_leaves/train',
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'archive/split_leaves/validation',
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical'
)

Found 2531 files belonging to 4 classes.
Found 633 files belonging to 4 classes.


In [3]:
# Rescale images
rescale = Rescaling(1./255)


# Channel Attention Module
def channel_attention(input_feature, ratio=8):
    channel = input_feature.shape[-1]
    
    shared_layer_one = Dense(channel // ratio, activation='relu', kernel_initializer='he_normal', use_bias=True)
    shared_layer_two = Dense(channel, kernel_initializer='he_normal', use_bias=True)
    
    avg_pool = GlobalAveragePooling2D()(input_feature)
    avg_pool = layers.Reshape((1, 1, channel))(avg_pool)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)
    
    max_pool = layers.GlobalMaxPooling2D()(input_feature)
    max_pool = layers.Reshape((1, 1, channel))(max_pool)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)
    
    cbam_feature = layers.Add()([avg_pool, max_pool])
    cbam_feature = layers.Activation('sigmoid')(cbam_feature)
    
    return Multiply()([input_feature, cbam_feature])

In [4]:
# Function to build the model with hyperparameters
def build_model(lr, dropout1, dropout2, dropout3):
    inputs = layers.Input(shape=(224, 224, 3))
    x = rescale(inputs)

    # First block
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = channel_attention(x)  
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropout1)(x)

    # Second block
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = channel_attention(x)  
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropout2)(x)

    # Third block
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = channel_attention(x)  
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropout3)(x)

    # Flatten and fully connected layers
    x = Flatten()(x)
    x = Dense(units=512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(units=256, activation='relu')(x)
    x = Dropout(0.5)(x)

    outputs = Dense(units=4, activation='softmax')(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    
    # Compile the model with given learning rate
    model.compile(optimizer=Adam(learning_rate=lr),  
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [5]:
# Firefly optimization for hyperparameters
class FireflyOptimization:
    def __init__(self, objective, bounds, n_fireflies=10, max_iter=10, alpha=0.5, beta=0.2, gamma=1.0):
        self.objective = objective
        self.bounds = bounds
        self.n_fireflies = n_fireflies
        self.max_iter = max_iter
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.fireflies = np.random.rand(n_fireflies, len(bounds))  # Random initialization
        
        # Scale fireflies to the bounds
        for i in range(len(bounds)):
            min_bound, max_bound = bounds[i]
            self.fireflies[:, i] = min_bound + self.fireflies[:, i] * (max_bound - min_bound)
        
        self.fitness = np.array([self.objective(firefly) for firefly in self.fireflies])
    
    def optimize(self):
        for t in range(self.max_iter):
            for i in range(self.n_fireflies):
                for j in range(self.n_fireflies):
                    if self.fitness[j] < self.fitness[i]:
                        distance = euclidean(self.fireflies[i], self.fireflies[j])
                        beta = self.beta * np.exp(-self.gamma * distance ** 2)
                        self.fireflies[i] += beta * (self.fireflies[j] - self.fireflies[i]) \
                                           + self.alpha * (np.random.rand(len(self.bounds)) - 0.5)
                        
                        # Clip firefly position to stay within bounds
                        for k in range(len(self.bounds)):
                            min_bound, max_bound = self.bounds[k]
                            self.fireflies[i, k] = np.clip(self.fireflies[i, k], min_bound, max_bound)
                        
                        # Recalculate fitness
                        self.fitness[i] = self.objective(self.fireflies[i])
            
            # Sort fireflies by fitness
            idx = np.argsort(self.fitness)
            self.fireflies = self.fireflies[idx]
            self.fitness = self.fitness[idx]
            
            print(f"Iteration {t+1}/{self.max_iter} | Best loss: {self.fitness[0]}")
        
        return self.fireflies[0], self.fitness[0]

In [6]:
# Define the objective function for optimization
def objective_function(params):
    lr, dropout1, dropout2, dropout3 = params
    
    # Build the model with the hyperparameters
    model = build_model(lr, dropout1, dropout2, dropout3)

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

    # Train the model for a small number of epochs to evaluate
    history = model.fit(train_dataset,
                        epochs=5,  # Limit for faster tuning
                        validation_data=validation_dataset,
                        callbacks=[early_stopping, lr_scheduler],
                        verbose=0)
    
    # Return the final validation loss
    val_loss = history.history['val_loss'][-1]
    return val_loss

In [ ]:
# Define search space for hyperparameters: [learning_rate, dropout1, dropout2, dropout3]
bounds = [(1e-5, 1e-2), (0.1, 0.6), (0.1, 0.6), (0.1, 0.6)]


# Run Firefly Optimization`
foa = FireflyOptimization(objective_function, bounds, n_fireflies=10, max_iter=10)
best_params, best_loss = foa.optimize()

print(f"Best parameters: {best_params}")
print(f"Best validation loss: {best_loss}")

# Final model with the best hyperparameters
best_lr, best_dropout1, best_dropout2, best_dropout3 = best_params
final_model = build_model(best_lr, best_dropout1, best_dropout2, best_dropout3)

In [ ]:
# Train the final model
final_history = final_model.fit(train_dataset,
                                epochs=20,
                                validation_data=validation_dataset,
                                callbacks=[early_stopping, lr_scheduler])

final_model.save('final_model_with_firefly.h5')